# Materials Data 

In [1]:
from gemd import MaterialTemplate, ProcessTemplate, MeasurementTemplate, ParameterTemplate, MaterialRun, MaterialSpec, RealBounds

In [2]:
from openmsimodel.science_kit.science_kit import ScienceKit
from openmsimodel.entity.gemd.material import Material
from openmsimodel.entity.gemd.process import Process
from openmsimodel.entity.gemd.measurement import Measurement
from openmsimodel.entity.gemd.ingredient import Ingredient
from openmsimodel.structures.materials_sequence import MaterialsSequence
from openmsimodel.db.open_db import OpenDB
from openmsimodel.graph.open_graph import OpenGraph
from openmsimodel.graph.helpers import launch_graph_widget

In [3]:
from pathlib import Path

Let's start by building GEMD templates.

In [4]:
temperature_measurement_template = MeasurementTemplate("Temperature")
polishing_process_template = ProcessTemplate("Polishing")
alloy_material_template = MaterialTemplate("Alloy")

You can interact with OpenMSIModel in many ways:

Let's wrap your GEMD templates with an entity of type GEMD Element (e.g., Material, Process, Measurement, Ingredient) class. It will create and associate the relevant specs and runs. 

In [5]:
alloy_ingredient = Ingredient("Alloy Ingredient")
polishing_process = Process("Polishing", template=polishing_process_template)
polished_alloy_material = Material("Polished Alloy", template=alloy_material_template)
temperature_measurement = Measurement('Pre-Heating Temperature', template=temperature_measurement_template)

Alternatively, you can build a GEMD Element object from a spec and/or run directly. It ensures the new correct links are made between GEMD objects, and that there is no duplication of specs or templates. 

In [6]:
name = "Alloy"
alloy_material_run = MaterialRun(name)
alloy_material_spec = MaterialSpec(name, template=alloy_material_template)
polished_alloy_material_2 = Material.from_spec_or_run(name, spec=alloy_material_spec, run=alloy_material_run) #

Your elements objects can be further composed into larger structures that fit different purposes.

A MaterialsSequence represent the natural and most common sequence of:
ingredients--(is used in a)-->process--(which outputs)-->material--(which generates)-->measurements.

Another example is MaterialsRepeatedSequence which can groups sequences that have the sample template across an entire graph, but doesn't necessarily helps build models. This shows that many kind of tools can be built around our wrapper, which abstract the initial GEMD layer. 

All of these tools, including but not only structures, exist inside your ScienceKit, which encompasses the entirety of your knowledge.

In [7]:
science_kit = ScienceKit()

polishing_sequence = MaterialsSequence(
    name=f"Polishing Alloy",
    science_kit=science_kit,
    ingredients=[alloy_ingredient],
    process=polishing_process,
    material=polished_alloy_material,
    measurements=[temperature_measurement],
)
polishing_sequence.link_within()

MaterialsSequence can also be formed from a spec and/or run, so from one GEMD object, you can reconstruct its logical neighbors of which there can be a large number (i.e., a sequence with many ingredients and measurements)

In [8]:
polishing_sequence_2 = MaterialsSequence.from_spec_or_run('Polishing Alloy', run=polished_alloy_material.run)

Now that we polished our alloy, let's heat it.

In [9]:
polished_alloy_ingredient = Ingredient("Polished Alloy Ingredient")
heating_process = Process("Heating", template=ProcessTemplate("Heating", parameters=ParameterTemplate(
    name="Temperature",
    bounds=RealBounds(0, 1500, "Kelvin"),
)))
heated_alloy = Material("Heated Alloy", template=alloy_material_template)
temperature_measurement = Measurement('Post-Heating Temperature', template=temperature_measurement_template)
heating_sequence = MaterialsSequence(
    name=f"Heating Alloy",
    science_kit=science_kit,
    material=heated_alloy,
    ingredients=[polished_alloy_ingredient],
    process=heating_process,
    measurements=[temperature_measurement],
)
heating_sequence.link_within() 
heating_sequence.link_prior(polishing_sequence, ingredient_name_to_link="Polished Alloy Ingredient")


as you can see above, MaterialsSequence can be linked between one another, so you can chain your workflow with more ease. 

Let's visualize what our graph looks like...

In [14]:
to_be_visualized = science_kit.assets()
output = str(Path().absolute() / "output")
open_graph = OpenGraph(name="Polishing,Heating", 
    science_kit=science_kit, 
    source=to_be_visualized, 
    output=output, 
    which='run', 
    dump_svg_and_dot=True)
G, relabeled_G, name_mapping = open_graph.build_graph(save=True)
launch_graph_widget(graph_source=science_kit.open_graphs["Polishing,Heating"].graphml_path, engine='yfiles') # ... you can pass 'G' or the path to the graphml file

Ignoring unexpected keyword argument in MeasurementSpec: properties
Ignoring unexpected keyword argument in MeasurementSpec: properties
Ignoring unexpected keyword argument in MeasurementSpec: properties
Ignoring unexpected keyword argument in MeasurementSpec: properties


-- Building runs of list with 44 items
Extracting list...
Done.
Relabeling nodes ...
Generating Graphviz version...
Saving graphs...
Dumping svg...
Time elapsed: 0.019192934036254883
Dumping dot...
Time elapsed: 0.000431060791015625
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/materials_data/output/Polishing,Heating_run.dot and /srv/hemi01-j01/openmsimodel/examples/materials_data/output/Polishing,Heating_run.svg 
Dumping graphml...
Time elapsed: 0.0012476444244384766
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/materials_data/output/Polishing,Heating_run.graphml
-- Analysis --
cycles in the graph: []
disregarded/total number of gemd objects: 0/44
number of connected components: 8
total nb of isolates in the graph: 0
Launching /srv/hemi01-j01/openmsimodel/examples/materials_data/output/Polishing,Heating_run.graphml


GraphWidget(layout=Layout(height='500px', width='100%'))

... and query some of its data from the database!

In [ ]:
db_name = "GEMD"
open_db = OpenDB(database_name=db_name, 
    science_kit=science_kit, 
    private_path="/home/arachid1/.private/gemd_db.json", 
    output=output)

table_name = "materials_data"
science_kit.open_dbs[db_name].custom_query("select top 3 context from gemdobject c where c.model_id={}".format(table_name))